# Database parsing

In [ ]:
import pandas, numpy as np
import parsingDatabaseUtils, re, os
import xml, itertools, collections, xml.etree.ElementTree as ET
import tqdm, importlib, dateparser, dateparser.search
import parseMeasurementsByDay

In [ ]:
pathDataIds = 'Venezolanas'
procedimientosDesc = pandas.read_csv(os.path.join(pathDataIds, 'procedimientosID.csv'), index_col = 0)
registrosByCaso = registros.groupby('Caso')

diagnosticos = pandas.read_csv(os.path.join(pathDataIds, 'diagnosis.csv'), index_col = 0)
#medicamentos = pandas.read_csv(os.path.join(pathDataIds, 'medicamentos.csv'), index_col = 0)

In [ ]:
pathFolder = 'Venezolanas'
casos = pandas.read_csv(pathFolder + '/casos.csv', index_col = 0)
pacientes = pandas.read_csv(pathFolder + '/pacientes.csv', index_col = 0)
pacientes.index = pacientes.index.map(str)

registros = pandas.read_csv(pathFolder + '/registros.csv', index_col = 0)
registros.index = registros.index.map(str)

#diagnosis = pandas.read_csv('Venezolanas2/diagnosis.csv', index_col = 0)
procedimientos = pandas.read_csv(pathFolder + '/procedimientos.csv', index_col = 0)
procedimientos.index = procedimientos.index.map(str)

entriesInfirmery = pandas.read_csv(pathFolder + '/enfermeriaMedidas.csv', index_col = 0)
entriesInfirmeryByCase = entriesInfirmery.groupby('IdAdmision')

In [ ]:
classificationProcedures = {'H2968': 'o', 'H2123' : 'o', 'H0165': 'o', 'H0193': 'o', 'H2120': 'o', 'H2379': 'o', 'H2383': 'o', 'H2386': 'o', 'H2407': 'o', 'H2415': 'o', 'H2595': 'o', 'H2684': 'o', 'H2849': 'o', 'H2852': 'o', 'H2880': 'o', 'H2882': 'o', 'H2884': 'o', 'H2892': 'o', 'H2901': 'o', 'H2904': 'o', 'H2910': 'o', 'H2916': 'o', 'H2959': 'o', 'H2963': 'o', 'H2974': 'a', 'H2975': 'a', 'H3038': 'o', 'H3065': 'o', 'H3066': 'o', 'H3078': 'o', 'H3089': 'p', 'H3092': 'p', 'H3094': 'p', 'H3099': 'a', 'H3100': 'a', 'H3108': 'o', 'H3109': 'o', 'H3111': 'o', 'H3114': 'o', 'H3118': 'o', 'H4421': 'o', 'H4494': 'o', 'H4496': 'o', 'HE020': 'o'}
classificationProcedures['H2962'] = 'o'
classificationProcedures.update({'H2938': 'o', 'H2952': 'o', 'H3030': 'o', 'H2392': 'o', 'H3046': 'o', 'H2982': 'o', 'H2834': 'o', 'H2966': 'o', 'H2937': 'o', 'H2885': 'o', 'H2948': 'o', 'H2408': 'o', 'H2911': 'o', 'H4372': 'o', 'H3008': 'o', 'H2981': 'o', 'H2847': 'o', 'H2840': 'o', 'H2965': 'o', 'H0004': 'o', 'H2410': 'o', 'H2271': 'o', 'H2977': 'o', 'H3052': 'o', 'H2955': 'o', 'H2927': 'o', 'H3031': 'o', 'H2835': 'o', 'H3082': 'o', 'H2946': 'o', 'H3002': 'o', 'H2932': 'o', 'H2913': 'o', 'H3085': 'o', 'H2274': 'o', 'H2918': 'o', 'H3077': 'o', 'H1099': 'o', 'H2953': 'o', 'H4447': 'o', 'H3029': 'o', 'H2964': 'o', 'H3014': 'o', 'H3075': 'o', 'H2705': 'o', 'H3059': 'o', 'H2888': 'o', 'H6783': 'o', 'H2853': 'o', 'H2382': 'o', 'H3116': 'o', 'H2883': 'o', 'H4495': 'o', 'H3070': 'o', 'H2957': 'o', 'H2970': 'o', 'H2221': 'o', 'H3067': 'o', 'H3032': 'o', 'H2121': 'o', 'H3001': 'o', 'H3113': 'o', 'H2874': 'o', 'H2954': 'o', 'H4417': 'o', 'H2926': 'o', 'H2582': 'o', 'H2917': 'o'})

## Select a case
Identify all  cases with an associated procedure

In [ ]:
# Identify all  cases with an associated procedure

In [ ]:
interventionToCase = {}
caseToProcedureBirths = {}
checkSeveralProcedures = []
unknownProc = []
procsNotFound = []
for i,r  in tqdm.tqdm_notebook(registros.iterrows()):
    if isinstance(r.RegistroXML, str) and '<row NombreCampo="IdDescripcion"' in r.RegistroXML:
        et = ET.fromstring(r.RegistroXML)
        idDescripcionProcedimiento = et.find('.//row[@NombreCampo="IdDescripcion"]').get('ValorCampo')
        if idDescripcionProcedimiento not in procedimientos.index:
            print('Procedimiento asociado con caso %s no se encuentra (# %s)' % (r.Caso, idDescripcionProcedimiento))
            continue
        interventionToCase[idDescripcionProcedimiento] = (r.Caso, r.NumeroHistoria, i)
        procedureType = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', procedimientos.loc[idDescripcionProcedimiento].XmlDescripcion)[0]
        try:
            typeProc = classificationProcedures[procedureType]
        except:
            typeProc = 'unknown'
            unknownProc.append(procedureType)
        if typeProc == 'p':
            
            #If it is the case that they need to stop labour because a c-section is needed
            if '<postDiagnosticoPrincipal>O821 - PARTO POR CESAREA DE EMERGENCIA</postDiagnosticoPrincipal>' in \
                procedimientos.loc[idDescripcionProcedimiento].XmlDescripcion:
                continue
                
            elif r.Caso in caseToProcedureBirths:
                r1 = caseToProcedureBirths[r.Caso]
                r2 =procedimientos.loc[idDescripcionProcedimiento]
                if r1.XmlDescripcion == r2.XmlDescripcion:
                    continue
                else:
                    checkSeveralProcedures.append(r.Caso)
                    print('error, varios procedimientos en caso: ', r.Caso)
                    if r1.FechaRegistro > r2.FechaRegistro:
                        caseToProcedureBirths[r.Caso] = procedimientos.loc[idDescripcionProcedimiento]
            else:
                caseToProcedureBirths[r.Caso] = procedimientos.loc[idDescripcionProcedimiento]

In [ ]:
import birthDatasetStructure
importlib.reload(birthDatasetStructure)
# Split in newborn
registrosByCaso = registros.groupby('Caso')
processedDatasets = {}
countCaseNotFound = 0
countPatientNotFound = 0

for c, p in tqdm.tqdm_notebook(caseToProcedureBirths.items()):
    if c not in casos.index:
        print('ERROR! Caso no encontrado', c)
        countCaseNotFound += 1
        continue
        
    
    if p.IdPaciente not in registrosByCaso.get_group(c).NumeroHistoria.values:
        print('ERROR! Pacientes inconsistente en caso', c, p.IdPaciente)
        countPatientNotFound += 1
        continue

    if str(p.IdPaciente) not in pacientes.index:
        print('ERROR! Pacientes no encontrado', p.IdPaciente)
        countPatientNotFound += 1
        break
        continue
     
    if c in entriesInfirmeryByCase.groups:
        entriesInf = entriesInfirmeryByCase.get_group(c)
    else:
        entriesInf = pandas.DataFrame()
    
    data = birthDatasetStructure.BirthDataset(c, casos.loc[c], p, registrosByCaso.get_group(c), 
                                                              pacientes,entriesInf)
    if data.epicrisis is not None:
        processedDatasets[c] = data

In [ ]:
def getAllDiagnosis(epicrisis):
    xml = ET.fromstring(epicrisis.RegistroXML)
    diagnosisEntrada = [parsingDatabaseUtils.findInXML('CodigoDiagnosticoRelacionado1',xml)]
    diagnosisEntrada += [parsingDatabaseUtils.findInXML('CodigoDiagnosticoRelacionado2',xml)]
    diagnosisEntrada += [parsingDatabaseUtils.findInXML('CodigoDiagnosticoRelacionado3',xml)]
    diagnosisEntrada += [parsingDatabaseUtils.findInXML('CodigoDiagnosticoPrincipal',xml)]
    
    diagnosisSalida = [parsingDatabaseUtils.findInXML('CodigoDiagnosticoRelacionado1Salida',xml)]
    diagnosisSalida += [parsingDatabaseUtils.findInXML('CodigoDiagnosticoRelacionado2Salida',xml)]
    diagnosisSalida += [parsingDatabaseUtils.findInXML('CodigoDiagnosticoRelacionado3Salida',xml)]
    diagnosisSalida += [parsingDatabaseUtils.findInXML('CodigoDiagnosticoPrincipalSalida',xml)]
    return list(map(lambda s: s.strip(), filter(lambda s:  s, diagnosisSalida +  diagnosisEntrada)))

In [ ]:
#Look for twins
wrongRegisters = []
print('Errors ! More than one register found')
for c, p in processedDatasets.items():
    if any((r  in getAllDiagnosis(p.epicrisis) for r in ['Z372', 'O300'])) :
        continue
    elif len(p.registrosRecienNacido) > 1 and  any((r  in getAllDiagnosis(p.epicrisis) for r in ['Z370'])) :
        wrongRegisters.append(c)
        print(c, getAllDiagnosis(p.epicrisis),  len(p.registrosRecienNacido))
        continue
    elif len(p.registrosRecienNacido) > 1:
        for k in p.registrosRecienNacido:
            if 'GEMEL' in parsingDatabaseUtils.findInXML('TexTarea_MotivoConsulta', p.registrosRecienNacido[k][k].RegistroXML):
                continue
        print(c, getAllDiagnosis(p.epicrisis),  len(p.registrosRecienNacido))        
        wrongRegisters.append(c)


In [ ]:
for c in wrongRegisters:
    del processedDatasets[c]

In [ ]:
import math
def relErrorInLimits(v1, v2, th = .05):
    try:
        v1 = float(v1)
        v2 = float(v2)
        relErr =  2 * math.fabs(v1 - v2)/(v1 + v2)
        return relErr < th
    except:
        return False
def joinDicts(d1, d2):
    d1 = d1.copy()
    d2 = d2.copy()
    for k in d1:
        if k in d2 and d1[k] != d2[k]:
            if relErrorInLimits(d1[k], d2[k]):
                d1[k] = (float(d1[k]) + float(d2[k]))/2
            else:
                d2['error_key_%s' % str(k)] = (True, (d1[k], d2[k]))
    d2.update(d1)
    return d2

In [ ]:
importlib.reload(parsingDatabaseUtils)
resSIP = {}
breakLoop = False
count = 0
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    if len(p.registrosRecienNacido) > 1:
        print('Warning! I still cannot handle twins')
        continue
    resSIP[c] = parsingDatabaseUtils.getMotherData(p)
    resSIP[c] = joinDicts(resSIP[c], parsingDatabaseUtils.getInformationFromProcedureDescription(p))
    for k in p.registrosRecienNacido:
        resSIP[c] = joinDicts(resSIP[c], parsingDatabaseUtils.getNewbornData(p, k))
    if len(p.registrosRecienNacido) == 1:
        resSIP[c]['VAR_0286'] = '0'
        resSIP[c]['VAR_0285'] = 'A'
df = pandas.DataFrame.from_dict(resSIP, orient = 'index')
#l = list(df[(df.VAR_0294 != df.VAR_0294) & (df.VAR_0293 != df.VAR_0293)].index)
#print(len(l))
print(count)

In [ ]:
import datetime
def toInt(s, d = 1):
    try:
        s = str(int(s))
        return addZeros(s, d)
    except:
        return "X"
def addZeros(s, d):
    s = str(s)
    return '0' * (d - len(str(s))) + s 

In [ ]:
#Quality control
n = len(df)
for c in df.columns:
    if sum(df[c] != df[c])/n > .01 and not c.startswith('error'):
        print(c, sum(df[c] != df[c])/n )

In [ ]:
import parseMeasurementsByDay
importlib.reload(parseMeasurementsByDay)
measurements = {}
measurementsControlsPrenatal = {}
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    measurements[c] = parseMeasurementsByDay.getParaClinicsHospitalisation(p) + parseMeasurementsByDay.getAllVitalSigns(p)
    if p.epicrisis is not None:
        measurementsControlsPrenatal[c] = parseMeasurementsByDay.parseParaclinicsBeforeHospitalisation(p.epicrisis)

In [ ]:
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    if len(p.registrosRecienNacido) > 1:
        print('Warning! I still cannot handle twins')
        continue

    res = {}
    dfMeasurements = pandas.DataFrame(data = measurements[c], columns =['Campo', 'Fecha', 'Valor'])
    dfMeasurements.Valor = dfMeasurements.Valor.map(lambda s: s.replace(',', '.') if isinstance(s, str) else s)
    dfMeasurements.Valor = dfMeasurements.Valor.astype(float)
    dfValues =dfMeasurements.groupby('Campo')['Valor'].agg(['median', 'max', 'min'])
    for var, row in dfValues.iterrows():
        res[var + '_median'] = row.median()
        res[var + '_max'] = row.max()
        res[var + '_min'] = row.min()
    resSIP[c].update(res)

In [ ]:
import parseDiagnosis
importlib.reload(parseDiagnosis)
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    if len(p.registrosRecienNacido) > 1:
        print('Warning! I still cannot handle twins')
        continue
    s = getAllDiagnosis(p.epicrisis)
    resSIP[c].update(parseDiagnosis.parseDiagnosis(s))

In [ ]:
dfResSIP = pandas.DataFrame.from_dict(resSIP, orient = 'index')

#Add SIP information
placeId = '806001061-8'
placeId = addZeros(placeId, 20)
dfResSIP['ID01'] = placeId + dfResSIP['VAR_0019'].map(lambda s: addZeros(s,20))  \
                                          + dfResSIP['VAR_0040'].map(lambda s: toInt(s, 2)) + \
                                        dfResSIP['VAR_0286'].map(toInt)
dfResSIP['VAR_0198'] = df['VAR_0198'].map(lambda s: str(int(s)) if s == s and s else s)
dfResSIP['VERSION'] = '4.1.2'
dfResSIP['FECHA'] = datetime.datetime.now().strftime("%Y-%m-%d")
dfResSIP['HORA'] = datetime.datetime.now().strftime("%H:%M:%S")
dfResSIP['USUARIO'] = 'CMRC-AUTO'

dfResSIP.to_csv('resultsVenezolanas.csv')

# Tests

From here on, it is 

In [ ]:
allDiagnosis = set()
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    allDiagnosis.update(set(getAllDiagnosis(p.epicrisis)))


In [ ]:
allDiagnosis = set(list(allDiagnosis) + list(parseDiagnosis.actionsDiagnosis.keys()))

In [ ]:
import parseDiagnosis

In [ ]:
for a in sorted(allDiagnosis):
    #if a in actionsDiagnosis:
    #    pass
    if a not in diagnosticos.index:
        continue 
    if a in parseDiagnosis.actionsDiagnosis:
        print("actionsDiagnosis['%s'] = '%s' #%s"% (a,parseDiagnosis.actionsDiagnosis[a], diagnosticos.loc[a, 'dfg']))
    else: 
        print("actionsDiagnosis['%s'] = '' #%s"% (a,diagnosticos.loc[a, 'dfg']))


In [ ]:
parsingDatabaseUtils.prettyPrintXML(p.epicrisis.RegistroXML)

In [ ]:
[p.epicrisis.Diagnostico, p.epicrisis.CodigoDiagnosticoRelacionado1,  p.epicrisis.CodigoDiagnosticoRelacionado2, p.epicrisis.CodigoDiagnosticoRelacionado3] 

In [ ]:
for c,  _ in df[df['sufrimientoFetal'] == 'SI'].iterrows():
    et = ET.fromstring(processedDatasets[c].epicrisis.RegistroXML)
    print(parsingDatabaseUtils.findInXML('MedicamentosAdministrado', et ))

In [ ]:
noEpicrsis = []
noIngreso = []

noDischarge = []
for c, p in processedDatasets.items():
    if p.epicrisis is None:
        noDischarge.append(c)
    if p.ingreso is None:
        noIngreso.append(c)
print('no emergency' , len(noIngreso), 'no epi', len(noDischarge), len(processedDatasets))

In [ ]:
pacientes.loc[str(casos.loc[noIngreso[0], 'Paciente'])], noIngreso[0], casos.loc[noIngreso[0]]

In [ ]:
#count = 0
#for c, p in processedDatasets.items():
#    if p.epicrisis is None:
#        continue
#    et = ET.fromstring(p.epicrisis.RegistroXML)
#    ant = parsingDatabaseUtils.fullCleanTxt(parsingDatabaseUtils.findInXML('AntecedentesHTML', et))
#    if not 'paracli' in ant:
#        count += 1
#    else:
#        print(ant)

In [ ]:
c = 'AD284225'
k = next(iter(processedDatasets[c].registrosRecienNacido.keys()))
parsingDatabaseUtils.prettyPrintXML(processedDatasets[c].registrosRecienNacido[k][k].RegistroXML)


In [ ]:
c = list(processedDatasets.keys())[884]
count = 0
for c in processedDatasets.keys():
    try:
        if 'ECLAMPS' in processedDatasets[c].epicrisis.RegistroXML:
            count += 1
            #parsingDatabaseUtils.prettyPrintXML(processedDatasets[c].epicrisis.RegistroXML)
    except:
        pass

In [ ]:
df.to_csv('test.csv')

In [ ]:
c = list(processedDatasets.keys())[244]
parsingDatabaseUtils.prettyPrintXML(processedDatasets[c].epicrisis.RegistroXML)

In [ ]:
import cProfile

In [ ]:
paraClinics = {}
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    if p.epicrisis is None:
        continue
    d =parseParaclinics(p.epicrisis)
    paraClinics[c] = paraclinicsToDF(parseParaclinics(p.epicrisis))


In [ ]:
dfParaClinics.noParaclinicalTestsConfirmed.value_counts()

In [ ]:
dfParaClinics = pandas.DataFrame.from_dict(paraClinics, orient = 'index')
dfParaClinics = dfParaClinics[sorted(dfParaClinics.columns)]
dfParaClinics.to_csv('paraclinics.csv')

In [ ]:
for p, d in processedDatasets.items():
    break

In [ ]:
getMedicamentsAndpProcedures(d.epicrisis)

In [ ]:
from html.parser import HTMLParser
pars =HTMLParser()

def splitByDate(txt):
    txt = pars.unescape(txt)
    txtByDate = txt.split('Fecha:')
    txtByDate = map(lambda s: s.strip(), txtByDate)
    txtByDate = {m.split()[0] : m for m in txtByDate if m}
            print(m)

    processedByDate = {}
    for v,m in txtByDate.items():
        processedByDate[v] =  re.findall('-([0-9a-zA-Z\- ]+) - (.+?)- Cantidad: ([0-9]+)', m, re.MULTILINE)
    return processedByDate
def getMedicamentsAndpProcedures(epicrisis):
    et = ET.fromstring(epicrisis.RegistroXML)
    medicamentos = parsingDatabaseUtils.findInXML('MedicamentosAdministrado' , et)
    procedimientos = parsingDatabaseUtils.findInXML('ProcedimientosAsociados' , et)
    #return medicamentos, procedimientos
    return splitByDate(medicamentos), splitByDate(procedimientos)

In [ ]:
parsingDatabaseUtils.prettyPrintXML(d.epicrisis.RegistroXML)

In [ ]:
for c, p in paraClinics.items():
    if len(p) != 1:
        print(paraClinics[c]['text'])
        print('-----------------')
        for k in p:
            if k != 'text':
                print(k, p[k])
        print('-----------------')

In [ ]:
def merge2DF(df1, df2):
    """
    Returns the merge of 2 df with the same index (1 to 1)
    In case of columns with the same name, check if the values are the same (or one of them is NA)
    Creates new error column to see where there has been problems
    """
    pass